### Preprocessing Eclipse Bug reports

#### Description of attributes
    Size of data: (103805, 13)
    Bug ID: Unique Identifier
    Important attributes: Product, Component, Assignee, Summary, Changed (possibly the frequency it has been changed with difference between opened and changed), Summary, Severity, Priority:
    Unimportant attributes: Status and Resolution. [We will not know them for the test samples]
    
    Summary has 103802 values i.e. 3 missing values will have to investigate them.


### Fixing of data formatting in the csv
    Some rows have shifted right by a few columns due the the data delimiting in the csv like rows 3613, 258
    That has been fixed and a new file total_fixed.csv has been saved

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/total.csv')

In [6]:
df.shape

(103805, 13)

In [17]:
import numpy as np

In [19]:
len(np.unique(df['Bug ID']))

103805

In [5]:
df.drop('Bug ID',axis=1,inplace=False).describe()

,Product,Component,Assignee,Status,Resolution,Summary,Changed,Priority,Severity,Keywords,Summary.1,Opened
count,103805,103805,103805,103805,103805,103802,103805,103805,103805,6814,103802,103805
unique,4,27,307,6,7,102339,88794,5,7,106,102339,96130
top,Platform,UI,jdt-ui-inbox@eclipse.org,RESOLVED,FIXED,#NAME?,9/27/05 9:12,P3,normal,performance,#NAME?,10/10/01 23:05
freq,60644,37694,3668,75695,53211,37,759,88902,71546,1272,37,61


In [10]:
df.loc[df.Summary.isnull(),:]

,Bug ID,Product,Component,Assignee,Status,Resolution,Summary,Changed,Priority,Severity,Keywords,Summary.1,Opened
538,1719,JDT,Debug,darin.eclipse@gmail.com,CLOSED,INVALID,NaN,11/2/01 8:56,P3,normal,NaN,NaN,10/10/01 22:18
26852,38524,Platform,SWT,platform-swt-inbox@eclipse.org,CLOSED,INVALID,NaN,6/5/03 13:04,P3,normal,NaN,NaN,6/5/03 11:35
44151,65246,JDT,UI,dirk_baeumer@ch.ibm.com,VERIFIED,FIXED,NaN,6/11/04 10:41,P3,normal,NaN,NaN,6/2/04 5:39


In [13]:
df.iloc[[257,258],:]

,Bug ID,Product,Component,Assignee,Status,Resolution,Summary,Changed,Priority,Severity,Keywords,Summary.1,Opened
3610,5706,JDT,Core,philippe_mulet@fr.ibm.com,RESOLVED,WORKSFORME,Cannot add two folders w/ same name but diff p...,2/7/02 8:46,P1,normal,NaN,Cannot add two folders w/ same name but diff p...,11/9/01 8:59
3611,8230,Platform,Debug,darin.eclipse@gmail.com,VERIFIED,FIXED,Console does not show process as <terminated>;...,2/7/02 9:14,P3,normal,NaN,Console does not show process as <terminated>;...,1/23/02 14:09


In [14]:
df.to_csv('../data/total_fixed.csv',index=False,header=True)

### Text preprocessing according to the Paper

1. Tokenization - word level
2. Stop-Word removal - 30 stop words: TIFIDF has that feature
3. Stemming - Porter Stemming.

This will be done only on the Summary attribute as others are categorical attributes.
